# Ch4. Multivariate Linear Regression

1. Simple Linear Regression 복습
2. Multivariate Linear Regression 이론
3. Naive Data Representation
4. Matrix Data Representation
5. Multivariate Linear  Regression
6. nn.module 소개
7. F.mse_loss 소개

## Simple Linear Regression

- 하나의 정보

$H(x) = Wx + b $

## Multivariate Linear Regression

- 여러 정보

$ H(x) = Wx + b $

## Hypothesis Function

$ H(x) = Wx + b $  
- x라는 vector와 W라는 matrix의 곱

$ H(x) = w_{1}x_{1} + w_{2}x_{2} + w_{3}x_{3} + b$
- 입력변수가 3개라면 weight도 3개!

## Hypothesis Function : Naive

In [ ]:
# H(x) 계산
hypothesis = x1_train * w1 + x2_train * w2 + x3_train *w3 + b

- 단순한 hypothesis 정의!
- But x가 길이 1000의 vector라면?

-> 변수가 굉장히 많아지면 일일이 쓰기엔 너무 많기에 Matrix 활용

## Hypothesis Function : Matrix

In [ ]:
# H(x) 계산
hypothesis = x_train.matmul(w) + b # or .mm or @

- matmul()로 한번에 계산
   - 더 간결
   - x의 길이가 바뀌어도 코드 변환 필요 없음
   - 속도가 더 빠름

$ H(x) = Wx + b$ 

## Cost Function : MSE

- 기존 simple Linear Regression 과 동일한 공식!  
$ cost(W) = \frac{1}{m}\sum_{i=1}^{m}(H(x^{(i)}) - y^{(i)})^{2} $   


- $ \frac{1}{m}\sum_{i=1}^{m} $ : Mean
- $ H(x^{(i)}) $ : Prediction
- $ y^{(i)} $ : Target

In [ ]:
cost = torch.mean( (hypothesis - y_train) ** 2)

## Gradient Descent with torch.optim

$ \nabla W = \frac{\delta cost}{\delta W} = \frac{2}{m}\sum_{i = 1}^{m}(Wx^{(i)} - y^{(i)})x^{(i)} $

$ W := W - \alpha \nabla W$

In [ ]:
# optimizer 설정
optimizer = optim.SGD([w,b], lr = 1e-5)

# optimizer 사용법
optimizer.zero_grad()
cost.backward()
optimizer.step()

## Full Code with torch.optim

In [5]:
import torch

# Data Definition 
x_train = torch.FloatTensor([ [73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]])

y_train = torch.FloatTensor([ [152], [185], [180], [196], [142]])

# model initialization
w = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad= True)

# optimizer 설정
optimizer = torch.optim.SGD([w,b], lr = 1e-5)

nb_epochs = 20

for epoch in range(nb_epochs + 1) :
    
    # H(x) 계산
    hypothesis = x_train.matmul(w) + b # or .mm or @
    
    # cost 계산
    cost = torch.mean( (hypothesis - y_train) **2)
    
    # cost로 H(x) 개선, gradient descent
    optimizer. zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis : {} Cost : {:.6f}'.format(
    epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))
    

Epoch    0/20 hypothesis : tensor([0., 0., 0., 0., 0.]) Cost : 29661.800781
Epoch    1/20 hypothesis : tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost : 9298.520508
Epoch    2/20 hypothesis : tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost : 2915.712402
Epoch    3/20 hypothesis : tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost : 915.040527
Epoch    4/20 hypothesis : tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost : 287.936005
Epoch    5/20 hypothesis : tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost : 91.371010
Epoch    6/20 hypothesis : tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost : 29.758139
Epoch    7/20 hypothesis : tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost : 10.445305
Epoch    8/20 hypothesis : tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost : 4.391228
Epoch    9/20 hypothesis : tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost : 2.49313

=> simple linear regression과 거의 같은 code가 의미하는 바는 pytorch의 확장성을 보여줌

- 점점 작아지는 Cost
- 점점  y에 가까워지는 H(x)
- Learning rate에 따라 발산할수도!

## nn.Module

In [ ]:
# model initialization
w = torch.zeros( (3,1), requires_grad = True)
b = torch.zeros( 1, requires_grad = True)

# H(x) 계산
hypothesis = x_train.matmul(w) + b # or .mm or @

import torch.nn as nn

class MultivariateLinearRegressionModel(nn.modeule) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x) :
        return self.linear(x)
    
hypothesis = model(x_train)

- nn.Module을 상속해서 model 생성
- nn.Linear(3,1)
   - 입력 차원 : 3
   - 출력 차원 : 1
- Hypothesis 계산은 forward() 에서!
- Gradient 계산은 PyTorch가 알아서 해준다. backward()

## F.mse_loss

In [ ]:
# cost 계산
cost = torch.mean( (hypothesis - y_train) ** 2)

import torch.nn.functional as F

# cost 계산
cost = F.mse_loss(prediction, y_train)
## bug 발생율이 줄어듬

- torch.nn.functional 에서 제공하는 loss function 사용
- 쉽게 다른 loss와 교체 가능! (ㅣ1_loss, smooth_l1_loss 등)

## Full Code with torch.optim(1)

In [11]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x) :
        return self.linear(x)

In [20]:
import torch

# data
x_train = torch.FloatTensor([ [73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]])

y_train = torch.FloatTensor([ [152], [185], [180], [196], [142]])

# model initialization
model = MultivariateLinearRegressionModel()
w, b = model.parameters()

# optimizer 설정
optimizer = torch.optim.SGD([w,b], lr = 1e-5)

nb_epochs = 20 
for epoch in range(nb_epochs + 1) :
    
    # H(x) 계산
    Hypothesis = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(Hypothesis, y_train)
    
    # cost로 계산 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis : {} Cost : {:.6f}'.format(
    epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))
    

Epoch    0/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 37060.980469
Epoch    1/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 11616.993164
Epoch    2/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 3641.650879
Epoch    3/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 1141.804932
Epoch    4/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 358.236725
Epoch    5/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 112.629311
Epoch    6/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 35.644527
Epoch    7/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 11.513758
Epoch    8/20 hypothesis : tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost : 3.950182
Epoch    9/20 hypothesis : tensor([152.8020, 183.6731, 180.967

- 지금까지 적은 양의 데이터를 가지고 학습함
- But deep learning은 많은 양의 data와 함께할 때 빛을 발함
- PyTorch에서는 많은 양의 data를 어떻게 다루는가?(다른 강의에서 진행 예정)

# Loading Data

- Data를 편하게 관리하는 방법

1. Multivariate Linear Regression 복습
2. "Minibatch" Gradient Descent 이론
3. PyTorch Dataset and DataLoader 사용법

## Multivariate Linear Regression

- 여러 정보(특징)으로부터 하나의 정보를 예측

$ H(x) = Wx + b $

In [ ]:
import torch

# data
x_train = torch.FloatTensor([ [73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]])

y_train = torch.FloatTensor([ [152], [185], [180], [196], [142]])

- 이전 강의에서 위 데이터를 활용하여 예측을 하였고
   - 점점  작아지는 Cost
   - 점점 y에 가까워지는 H(x)
   - Learning rate에 따라 발산 할 수도  있음

### Data in the Real World

- 복잡한 ML model을 학습하려면 엄청난 양의 데이터가 필요하다.
- 대부분 data set은 적어도 수십만 개의 데이터를 제공

### Data in the Real World : Problem

- 엄청난 양의 데이터를 한 번에 학습시킬 수 없다!
   - 너무 느림
   - Hardware 상 불가능
   
- 그렇다면 일부분의 data로만 학습하면 어떨까?

=> 그렇게 나온 IDEA가 Minibatch

## Minibatch Gradient Descent

- 전체 data를 균일하게 나눠서 학습

## Minibatch Gradient Descent : Effects

- update를 좀 더 빠르게 가능
- 전체 데이터를 쓰지 않아서 잘못된 방향으로 update를 할 수도 있음

## PyTorch Data Set

In [21]:
# torch.utils.data.Dataset 상송
from torch.utils.data import Dataset

class CustomDataset(Dataset) :
    def __init__(self) :
        self.x_data = [ [73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]]
        self.y_data = [ [152], [185], [180], [196], [142]]
    
    # 이 data set의 총 data 수
    def __len__(self) :
        return len(self.x_data)
    
    # 어떠한 index를 받았을때, 그에 상응하는 입출력 data 반환
    def __getitem__(self, idx) :
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x,y
    
dataset = CustomDataset()

## PyTorch DataLoader

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    
            dataset,
    
            # 각 minibatch의 크기, 통상적으로 2의 제곱수로 설정
            batch_size = 2,
    
            #Epoch 마다 data set을 섞어서, data가 학습되는 순서를 바꾼다.
            shuffle = True,
            )

## Full Code with Dataset And DataLoader

In [ ]:
nb_epochs = 20

for epoch in range(nb_epochs + 1 ) :
    # enumerate(dataloader) : minibatch index & data
    for batch_idx, samples in enumerate(dataloader) :
        x_train, y_train = samples
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch : {} Cost : {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()))
        # len(dataloader) : 한 epoch당 batch 개수

=> 어떠한 숫자 하나를 예측하는 모델      
=> 다음은 분류 모델